<a href="https://colab.research.google.com/github/walnashgit/S30-Capstone/blob/main/FineTunePhi2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

walnash_llava_instruct_150k_path = kagglehub.dataset_download('walnash/llava-instruct-150k')
walnash_coco2014embeddingsm2_path = kagglehub.dataset_download('walnash/coco2014embeddingsm2')
walnash_visionprojectorphi2_m2_pytorch_default_1_path = kagglehub.model_download('walnash/visionprojectorphi2-m2/PyTorch/default/1')

print('Data source import complete.')


In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install -U bitsandbytes

In [ ]:
cfg = dict(
    vision_projector_file='/kaggle/input/visionprojectorphi2-m2/pytorch/default/1/projectionModel_ckpt_0_m2.pth',
    checkpoint_dir='/kaggle/working/checkpoint',
    finetuned_dir='/kaggle/working/fine_tuned',
    resume=False,
    inference=False,
    un_tuned=False,
    raw_test=False,
    batch_size=16,
    clip_dim=768, #512,
    phi_dim=2560,
    image_token='<|image|>',
    image_token2="<image>",
)

In [ ]:
import os

WORKING_DIR = '/kaggle/working'
CHECKPOINT_DIR = os.path.join(WORKING_DIR, cfg['checkpoint_dir'])
FINE_TUNED_DIR = os.path.join(WORKING_DIR, cfg['finetuned_dir'])

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(FINE_TUNED_DIR, exist_ok=True)

print(f"Checkpoint directory created at: {CHECKPOINT_DIR}")
print(f"Fine-tuned model directory created at: {FINE_TUNED_DIR}")

Checkpoint directory created at: /kaggle/working/checkpoint
Fine-tuned model directory created at: /kaggle/working/fine_tuned


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

class ProjectionLayer(torch.nn.Module):
    def __init__(self, clip_dim, phi_dim):
        super().__init__()
        self.linear = torch.nn.Linear(clip_dim, phi_dim)

    def forward(self, x):
        return self.linear(x)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "dense",
        "fc1",
        "fc2"
    ]
)

IGNORE_INDEX = -100

class PhiWithVision(torch.nn.Module):
    def __init__(self, device='cuda', tokenizer=None, tokenizer_length=None):
        super().__init__()
        clip_dim = cfg['clip_dim']
        phi_dim = cfg['phi_dim']
        self.device = device
        self.tokenizer = tokenizer
        self.loss_fct = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX, label_smoothing=0.1)

        # for inference
        if cfg['un_tuned'] or cfg['raw_test']:
            self.phi_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",
                                                                  low_cpu_mem_usage=True,
                                                                  return_dict=True,
                                                                  trust_remote_code=True,
                                                                  torch_dtype=torch.float32)
            self.phi_model.eval()
        elif cfg['inference']:
            base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",
                                                                  low_cpu_mem_usage=True,
                                                                  return_dict=True,
                                                                  trust_remote_code=True,
                                                                  torch_dtype=torch.float32)

            if tokenizer_length is not None:
                base_model.resize_token_embeddings(tokenizer_length)
            self.phi_model = PeftModel.from_pretrained(base_model, cfg['finetuned_dir'])
            self.phi_model.merge_and_unload()
            self.phi_model.eval()
        else:
            if device == 'mps':
                self.phi_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",
                                                                      trust_remote_code=True)
                self.phi_model = self.phi_model.half()
            else:
                self.phi_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",
                                                                      trust_remote_code=True,
                                                                      quantization_config=bnb_config,
                                                                      low_cpu_mem_usage=True)
            if tokenizer_length is not None:
                self.phi_model.resize_token_embeddings(tokenizer_length)
            self.phi_model.config.use_cache = False
            self.phi_model = prepare_model_for_kbit_training(self.phi_model)
            self.phi_model = get_peft_model(self.phi_model, peft_config)
            self.phi_embeddings = self.phi_model.get_input_embeddings()
            self.phi_model.train()

        if cfg['vision_projector_file'] == './checkpoint/vp_ckpt_0.pth':
            self.projectionModel = ProjectionLayer2(clip_dim, phi_dim)
        else:
            self.projectionModel = ProjectionLayer(clip_dim, phi_dim)

        # if cfg['raw_test']:
        if device == 'cuda':
            # trained_proj_model = torch.load(cfg['vision_projector_file'], map_location='cpu')
            trained_proj_model = torch.load(cfg['vision_projector_file'])
            self.projectionModel.load_state_dict(trained_proj_model['model_state_dict'])
            self.projectionModel.to(device)
        else:
            # trained_proj_model = torch.load(cfg['vision_projector_file'])
            trained_proj_model = torch.load(cfg['vision_projector_file'], map_location='cpu')
            self.projectionModel.load_state_dict(trained_proj_model['model_state_dict'])

        # freeze vision projection
        self.projectionModel.eval()
        for param in self.projectionModel.parameters():
            param.requires_grad = False

        # self.fusion_layer = torch.nn.Linear(phi_dim * 2, phi_dim)

    def forward(self, input_ids=None, attention_mask=None, image_embedding=None, image_token_positions=None, labels=None, **kwargs):
        if cfg['raw_test']:
            # Generate text based only on the image embedding
            # projected_embed = self.projectionModel(image_embedding)
            # inputs_embeds = projected_embed.unsqueeze(1)

            inputs_embeds = self.projectionModel(image_embedding)
            if cfg['clip_dim'] == 512:
                inputs_embeds = inputs_embeds.unsqueeze(1)

            # Create a dummy input_ids tensor
            batch_size = inputs_embeds.shape[0]
            input_ids = torch.full((batch_size, 1), self.tokenizer.bos_token_id, dtype=torch.long,
                                   device=inputs_embeds.device)


            # Prepare generation config
            generation_kwargs = {
                'input_ids': input_ids,
                'max_length': 50,
                # 'inputs_embeds': inputs_embeds,
                **kwargs  # This will include all other passed parameters
            }

            with torch.no_grad():
                outputs = self.phi_model(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
            return outputs
        else:
#             print(f"Image embedding range: {image_embedding.min().item():.4f} to {image_embedding.max().item():.4f}")

            projected_clip = self.projectionModel(image_embedding).requires_grad_(requires_grad=False)

            combined_input, new_labels = self.prepare_input_embed(input_ids, projected_clip, image_token_positions, labels)
            phi_outputs = self.phi_model(inputs_embeds=combined_input, attention_mask=attention_mask)

            if torch.isinf(phi_outputs.logits).any():
                print("WARNING: inf values in logits!")
            if torch.isnan(phi_outputs.logits).any():
                print("WARNING: nan values in logits!")

#             return phi_outputs, new_labels
            logits = phi_outputs.logits
            X = logits[:, :-1, :].contiguous()
            Y = new_labels[:, 1:].contiguous().to(self.device, dtype=torch.long)
            X = X.view(-1, X.size(-1))
            Y = Y.view(-1)

            loss = self.loss_fct(X, Y)
            return loss

    def prepare_input_embed(self, input_ids, projected_clip, image_token_pos, labels):
        new_input_embeds = []
        new_labels = []
        for batch_idx, cur_input_ids in enumerate(input_ids):

            pos = image_token_pos[batch_idx]
            phi_text_embedding = self.phi_embeddings(cur_input_ids)
            # if pos.numel() > 0 and pos.item() < seq_length: # lets assume image token pos is present. It should be
            cur_new_input_embeds = [phi_text_embedding[:pos], projected_clip[batch_idx], phi_text_embedding[pos + 1:]]
            cur_new_input_embeds = [x.to(self.device) for x in cur_new_input_embeds]
            cur_new_input_embeds = torch.cat(cur_new_input_embeds, dim=0)
            new_input_embeds.append(cur_new_input_embeds)

            if labels is not None:
                cur_labels = labels[batch_idx]
                cur_new_labels = [cur_labels[:pos],
                                  torch.full((projected_clip[batch_idx].shape[0],), IGNORE_INDEX, device=labels.device,
                                             dtype=labels.dtype), cur_labels[pos + 1:]]

                cur_new_labels = torch.cat(cur_new_labels, dim=0)
                new_labels.append(cur_new_labels)

        new_input_embeds = torch.stack(new_input_embeds, dim=0)
        new_labels = torch.stack(new_labels, dim=0)
        return new_input_embeds, new_labels

In [ ]:
import json
import random

import torch
from torch.utils.data import Dataset, Sampler
import h5py



class MultiModalLlavaDataset(Dataset):
    def __init__(self, embedding_file, instruct_file, tokenizer, max_length=128):
        self.embedding_file = h5py.File(embedding_file, 'r')
        self.instruct_file = instruct_file
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.image_ids = set(self.embedding_file.keys())
        self.conversation_data = self.get_conversation_data()  # List of text inputs

        # Special token for image placeholder
        self.image_token = cfg['image_token2']
        # Add the image token to the tokenizer
        # self.tokenizer.add_special_tokens({'additional_special_tokens': [self.image_token]})

    def __len__(self):
        return len(self.conversation_data)

    def __getitem__(self, idx):
        conv_item = self.conversation_data[idx]
        image_id = str(conv_item.get('image_id'))

        image_embedding = torch.tensor(self.embedding_file[image_id][()])
        conv = conv_item.get('qna')
        # prepared_conv = self.prepare_conversation(conversations)
        prompt = '<image>\n' + conv

        # Tokenize conversation
        tokenized_prompt = self.tokenizer(prompt, truncation=True, max_length=self.max_length, padding='max_length')
        token_ids = tokenized_prompt['input_ids']

        input_pad_tokens = self.max_length - (len(token_ids) + image_embedding.size(0) - 1)

        if input_pad_tokens < 0:
            input_pad_tokens = 0
            # conn_tokens = self.tokenizer.encode('AI### ')
            # token_ids = tokenizer_image_token(parts[0], tokenizer=self.tokenizer)
            truncate_len = self.max_length - (image_embedding.size(0) - 1)
            token_ids = token_ids[:truncate_len]

        input_ids = torch.cat(
            [
                torch.tensor(token_ids, dtype=torch.int32),
                torch.tensor([self.tokenizer.pad_token_id] * input_pad_tokens, dtype=torch.int32)
            ],
            dim=0
        )

        labels = input_ids.clone()
        parts = prompt.split('AI### ')
        if len(parts) != 2:
            print(prompt)
            raise Exception("Not proper QnA text: " + conv)

        que_len = len(self.tokenizer(parts[0] + 'AI### '))
        labels[0: que_len] = IGNORE_INDEX

        image_token_id = self.tokenizer.convert_tokens_to_ids(self.image_token)
        # image_token_position = (torch.tensor(tokenized['input_ids']) == image_token_id).nonzero(as_tuple=True)[0]
        image_token_position = (input_ids == image_token_id).nonzero(as_tuple=True)[0]

        return {
            'image_embedding': image_embedding,
            'input_ids': input_ids,
            # 'attention_mask': torch.tensor(tokenized['attention_mask']),
            'image_token_position': image_token_position,
            'labels': labels
        }

    def get_conversation_data(self):
        with open(self.instruct_file, 'r') as f:
            instruct_file_json = json.load(f)
            return self.split_conversation(instruct_file_json)

    def split_conversation(self, instruct_file_json):
        instruct_data = []
        seps = ['\n', '<|endoftext|>']
        for conv_dict in instruct_file_json:
            image_id = self.remove_leading_zeros(str(conv_dict.get('id')))
            conv = conv_dict.get('conversations')
            if image_id in self.image_ids:
                t = None
                for i, qa in enumerate(conv):
                    role = qa['from']
                    msg = qa['value'].replace('<image>', '')
                    if i % 2 == 0:
                        t = ''

                        if role == 'human':
                            t += 'Human### ' + msg + seps[0]
                    else:
                        if role == 'gpt' and t and msg:
                            t += 'AI### ' + msg + seps[1]

                        if t:
                            instruct_dict = dict(
                                image_id=image_id,
                                qna=t
                            )
                            instruct_data.append(instruct_dict)

        return instruct_data

    def remove_leading_zeros(self, number_string):
        return number_string.lstrip('0') or '0'

    def prepare_conversation(self, conversations):
        # Prepare conversation
        conversation = ""
        for turn in conversations:
            if turn['from'] == 'human':
                conversation += f"Human###: {turn['value']}\n"
            else:
                conversation += f"AI###: {turn['value']}{self.tokenizer.eos_token}"
        return conversation


class BucketBatchSampler(Sampler):
    def __init__(self, data_source, batch_size):
        super().__init__(data_source)
        self.data_source = data_source
        self.batch_size = batch_size
        self.buckets = self._create_buckets()

    def _create_buckets(self):
        # Sort data by sequence length and create buckets
        # This is a simplified example; adjust based on your actual data structure
        sorted_data = sorted(range(len(self.data_source)),
                             key=lambda idx: len(self.data_source[idx]['input_ids']))
        return [sorted_data[i:i + self.batch_size] for i in range(0, len(sorted_data), self.batch_size)]

    def __iter__(self):
        random.shuffle(self.buckets)
        for bucket in self.buckets:
            yield bucket

    def __len__(self):
        return len(self.buckets)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import itertools
from torch.nn.utils.rnn import pad_sequence
import gc

phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
phi_tokenizer.pad_token = phi_tokenizer.eos_token
phi_tokenizer.add_special_tokens({'additional_special_tokens': [cfg['image_token2']]})

def fine_tune(model, dataloader, num_epochs, device):
    model.train()
#     optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

#     loss_fct = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX, label_smoothing=0.1)
    max_grad_norm = 1.0

    step_count = 0
    prev_loss = 1000
    bestLoss = 0
    bestStep = 0
    accumulation_steps = 2
    use_amp = True
    optimizer.zero_grad()
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    for epoch in range(num_epochs):
        total_loss = 0
        step_loss = 0
        batch_iterator = tqdm(dataloader, desc=f"Processing Epoch {epoch:02d}")

        for batch in batch_iterator:

        # for batch in itertools.islice(batch_iterator, 2):
            input_ids = batch['input_ids'].to(device)
            # attention_mask = batch['attention_mask'].to(device)
            image_embedding = batch['image_embedding'].to(device)
            image_token_positions = batch['image_token_position'].to(device)
            labels = batch['labels'].to(device)
            with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
#             with torch.autocast(device_type=device, dtype=torch.float32):
                # Forward pass
#                 outputs, new_labels = model(input_ids, None, image_embedding, image_token_positions, labels)
                loss = model(input_ids, None, image_embedding, image_token_positions, labels)

#             if isinstance(outputs, tuple):
#                 logits = outputs[0]
#             elif hasattr(outputs, 'logits'):
#                 logits = outputs.logits
#             else:
#                 raise ValueError("Unexpected output format from the model")

#             X = logits[:, :-1, :].contiguous()
#             Y = new_labels[:, 1:].contiguous().to(device, dtype=torch.long)

#             # Add debugging info
# #             print(f"\nShapes before view - X: {X.shape}, Y: {Y.shape}")
# #             print(f"Number of IGNORE_INDEX tokens: {(Y == -100).sum().item()}")


#             X = X.view(-1, X.size(-1))
#             Y = Y.view(-1)

            # Add value checks
#             print(f"Logits range: {X.min().item():.4f} to {X.max().item():.4f}")
#             print(f"Labels range: {Y.min().item()} to {Y.max().item()}")

#             # Calculate loss
#             loss = loss_fct(X, Y)

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()
#             step_loss += loss.item()

            try:
#                 loss = loss_fct(X, Y) / accumulation_steps

                if torch.isinf(loss) or torch.isnan(loss):
                    print("WARNING: Loss is inf/nan! Skipping batch")
#                     continue

#                 loss.backward()
                scaler.scale(loss).backward()

                # Add gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                if step_count > 0 and (step_count + 1) % accumulation_steps == 0:
#                     optimizer.step()
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()

#                 optimizer.step()

                total_loss += loss.item()
                step_loss += loss.item()

            except RuntimeError as e:
                print(f"Error in loss computation: {str(e)}")
                print(f"X shape: {X.shape}, Y shape: {Y.shape}")
                print(f"X dtype: {X.dtype}, Y dtype: {Y.dtype}")
                continue

#             print(f"\n Epoch {epoch + 1}, step: {step_count}, Loss: {loss.item()}, total loss: {total_loss}")

            if loss.item() < prev_loss:
                bestLoss = loss.item()
                bestStep = step_count
                print(f"\n Epoch {epoch + 1}, step: {step_count}, Loss: {loss.item()}, total loss: {total_loss}")
                save_model(epoch, model, total_loss, optimizer, step_count)
                prev_loss = loss.item()
            elif (step_count > 0 and step_count % 10 == 0):
                print(f"\n Epoch {epoch + 1}, step: {step_count}, loss: {loss.item()}, total loss: {total_loss}")
                if device == 'cuda':
                    print('clearing cache')
                    gc.collect()
                    torch.cuda.empty_cache()


            # Clear cache periodically
            if step_count % 10 == 0:
                if device == 'cuda':
                    torch.cuda.empty_cache()

            step_count += 1

#         if device == 'cuda':
#             torch.cuda.empty_cache()
#         elif device == 'mps':
#             torch.mps.empty_cache()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss}")


def save_model(epoch, model, loss, optimizer, step_count):
    print('saving model')
    torch.save({
        'epoch': epoch,
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'step_count': step_count,
    }, '%s/phiModel_ckpt_%s.pth' % (cfg['checkpoint_dir'], epoch))

    model.phi_model.save_pretrained(cfg['finetuned_dir'])
    phi_tokenizer.save_pretrained(cfg['finetuned_dir'])


def run_fine_tune():
    cfg['un_tuned'] = False
    cfg['inference'] = False
    torch.manual_seed(35)

    device = 'cpu'
    if torch.cuda.is_available():
        print('using CUDA')
        device = 'cuda'
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        device = 'mps'
        torch.mps.empty_cache()


    embedding_file = '/kaggle/input/coco2014embeddingsm2/coco2014_clip_embeddings_m2.h5'
    instruction_file = '/kaggle/input/llava-instruct-150k/llava_instruct_150k.json'

    batch_size = 12
    dataset = MultiModalLlavaDataset(embedding_file,
                                     instruction_file, phi_tokenizer)
    sampler = BucketBatchSampler(dataset, batch_size)
    dataloader = DataLoader(dataset, batch_sampler=sampler, collate_fn=collate_fn)

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        device = 'mps'
        torch.mps.empty_cache()

    torch.manual_seed(25)

    # model = PhiWithVision.load_from_checkpoint(model_path)
    model = PhiWithVision(device=device, tokenizer_length=len(phi_tokenizer))
    model.to(device)

    # model.phi_model.resize_token_embeddings(len(phi_tokenizer))

    fine_tune(model, dataloader, 1, device)


def collate_fn(batch):
    # Separate the different items in the batch
    image_embeddings = [item['image_embedding'] for item in batch]
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    # attention_masks = [torch.tensor(item['attention_mask']) for item in batch]
    image_token_positions = [item['image_token_position'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=phi_tokenizer.pad_token_id)

    image_embeddings = torch.stack(image_embeddings)
    labels = torch.stack(labels)

    image_token_positions = torch.stack(image_token_positions)

    return {
        'image_embedding': image_embeddings,
        'input_ids': input_ids_padded,
        # 'attention_mask': attention_masks_padded,
        'image_token_position': image_token_positions,
        'labels': labels
    }

In [ ]:
run_fine_tune()

using CUDA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_30/3294830626.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_proj_model = torch.load(cfg['vision_projector_file'])
/tmp/ipykernel_30/462785546.p


 Epoch 1, step: 0, Loss: 6.850039005279541, total loss: 6.850039005279541
saving model


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:258: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Processing Epoch 00:   0%|          | 1/18148 [00:06<34:11:20,  6.78s/it]

...


 Epoch 1, step: 8650, loss: 2.776125431060791, total loss: 20767.941595435143
clearing cache


Processing Epoch 00:  48%|████▊     | 8661/18148 [7:33:54<8:21:47,  3.17s/it]


 Epoch 1, step: 8660, loss: 2.1857335567474365, total loss: 20790.802763819695
clearing cache


Processing Epoch 00:  48%|████▊     | 8666/18148 [7:34:10<8:16:55,  3.14s/it]


KeyboardInterrupt: 

In [ ]:
import shutil
from IPython.display import FileLink, display

# Define the paths
WORKING_DIR = '/kaggle/working'
FINE_TUNED_DIR = os.path.join(WORKING_DIR, 'fine_tuned')
ARCHIVE_NAME = 'fine_tuned_model1.tar.gz'
ARCHIVE_PATH = os.path.join(WORKING_DIR, ARCHIVE_NAME)

# Function to create a downloadable archive
def create_downloadable_archive():
    if os.path.exists(FINE_TUNED_DIR):
        # Create a tar.gz archive of the fine_tuned directory
        shutil.make_archive(
            os.path.join(WORKING_DIR, 'fine_tuned_model1'),
            'gztar',
            FINE_TUNED_DIR
        )
        print(f"Archive created: {ARCHIVE_PATH}")

        # Create a download link
        display(FileLink(ARCHIVE_PATH, result_html_prefix="Click here to download the fine-tuned model: "))
    else:
        print(f"Error: {FINE_TUNED_DIR} does not exist.")



In [ ]:
# Call this function after your model training is complete and you've saved the model
create_downloadable_archive()

# # Optional: Print the contents of the archive for verification
# print("\nContents of the archive:")
# !tar -tvf {ARCHIVE_PATH}

Archive created: /kaggle/working/fine_tuned_model1.tar.gz


/kaggle/working/fine_tuned_model1.tar.gz